## Imports

In [33]:
import pandas as pd
import yaml
from pytorch_lightning import Trainer, seed_everything
from eyemind.trainer.loops import KFoldLoop
from eyemind.models.classifier import EncoderClassifierMultiSequenceModel
from eyemind.models.encoder_decoder import VariableSequenceLengthEncoderDecoderModel
from eyemind.dataloading.gaze_data import SequenceToLabelDataModule, SequenceToSequenceDataModule

## Fixation Identification

## Comprehension Classification

In [3]:
name_to_cls = {"trainer": Trainer, "model": EncoderClassifierMultiSequenceModel, "data": SequenceToLabelDataModule}
def instantiate_class_from_config(config, cls_name):
    args = config[cls_name]
    return name_to_cls[cls_name](**args)

def instantiate_lightningmodules(config):
    modules = [instantiate_class_from_config(config, k) for k in name_to_cls.keys()]
    return tuple(modules)

In [4]:
configpath = "/Users/rickgentry/emotive_lab/eyemind/experiment_configs/inf_multiseq_config.yml"

In [5]:
with open(configpath, 'r') as f:
    config = yaml.safe_load(f)

In [6]:
seed_everything(42, workers=True)
trainer, model, data = instantiate_lightningmodules(config)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CNN Layers: [16, 32]
16 2
32 16


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


### Testing reproducibility of folds

In [7]:
data.setup()

In [8]:
data.train_dataset

In [9]:
data.save_setup("setup_ROTE_X.yml")

In [10]:
loaded_data = instantiate_class_from_config(config, "data")

In [11]:
loaded_data.load_setup_path = "setup_ROTE_X.yml"
loaded_data.setup()

In [12]:
assert (set(loaded_data.train_dataset.indices) == set(data.train_dataset.indices))
assert (set(loaded_data.test_dataset.indices) == set(data.test_dataset.indices))

In [13]:
train_test_split = (data.train_dataset.dataset.get_files_from_indices(data.train_dataset.indices), data.test_dataset.dataset.get_files_from_indices(data.test_dataset.indices))

In [14]:
data.setup_folds(4)

In [15]:
data.save_folds("folds_rote_x.yml")

In [16]:
loaded_data.load_folds("folds_rote_x.yml")

In [17]:
[set(split_data[0]) == set(loaded_split[0]) for split_data, loaded_split in zip(data.splits, loaded_data.splits)]

[True, True, True, True]

In [18]:
file_folds = [(data.train_dataset.dataset.get_files_from_indices(split[0]),data.train_dataset.dataset.get_files_from_indices(split[1]) ) for split in data.splits]

In [19]:
file_folds

[(['EML1_087-Bias8.csv',
   'EML1_049-Validity8.csv',
   'EML1_069-Variables3.csv',
   'EML1_116-Hypotheses8.csv',
   'EML1_070-Variables8.csv',
   'EML1_018-Bias7.csv',
   'EML1_033-Validity1.csv',
   'EML1_069-Hypotheses5.csv',
   'EML1_025-CausalClaims6.csv',
   'EML1_125-CausalClaims6.csv',
   'EML1_020-Hypotheses1.csv',
   'EML1_111-CausalClaims6.csv',
   'EML1_048-Validity6.csv',
   'EML1_021-CausalClaims2.csv',
   'EML1_029-Variables5.csv',
   'EML1_083-Hypotheses8.csv',
   'EML1_015-Validity6.csv',
   'EML1_115-Validity6.csv',
   'EML1_136-Bias6.csv',
   'EML1_005-CausalClaims8.csv',
   'EML1_042-Validity6.csv',
   'EML1_103-CausalClaims8.csv',
   'EML1_106-Bias2.csv',
   'EML1_071-CausalClaims6.csv',
   'EML1_050-Variables8.csv',
   'EML1_072-Bias6.csv',
   'EML1_050-Bias2.csv',
   'EML1_012-Hypotheses8.csv',
   'EML1_030-Hypotheses8.csv',
   'EML1_036-Hypotheses4.csv',
   'EML1_036-CausalClaims6.csv',
   'EML1_014-CausalClaims3.csv',
   'EML1_036-Bias6.csv',
   'EML1_087-Hypo

In [20]:
def write_splits(splits, filepath, types="train_test"):
    if types == "train_test":
        split_out = {"train": splits[0], "test": splits[1]}
    else:
        #split_dict = {f"fold{i}": {"train": split[0], "val": split[1]} for i,split in enumerate(splits) }
        split_out = {"folds": [{"train": split[0], "val": split[1]} for split in splits]}
    with open(filepath, 'w') as f:
        yaml.dump(split_out, f)
    return splits

def load_file_folds(path):
    with open(path, 'r') as f:
        file_folds_dict = yaml.safe_load(f)    
    return [(fold["train"], fold["val"]) for fold in file_folds_dict["folds"]]

def compare_splits(filepath, splits, types="train_test"):
    with open(filepath, "r") as f:
        file_splits = yaml.safe_load(f)
    if types == "train_test":
        assert (len(splits[0]) == len(file_splits['train']))
        assert (set(splits[0]) == set(file_splits['train']))
    else:
        for i, split in enumerate(splits):
            for j,ds_type in enumerate(("train", "val")):
                #print(f"List Lengths: {len(split[i])}, {len(file_splits[f'fold{i}'][ds_type])}")
                file_set = set(file_splits[f"fold{i}"][ds_type])
                split_set = set(split[j])
                #print(len(split_set), len(file_set))
                assert(len(split_set) == len(file_set))
                assert(split_set == file_set)



In [21]:
# write_splits(train_test_split, "/Users/rickgentry/emotive_lab/eyemind/data/inference_train_test_rotex_splits.yml", "train_test")
write_splits(file_folds, "/Users/rickgentry/emotive_lab/eyemind/data/inference_train_val_rotex_folds.yml", "folds")

[(['EML1_087-Bias8.csv',
   'EML1_049-Validity8.csv',
   'EML1_069-Variables3.csv',
   'EML1_116-Hypotheses8.csv',
   'EML1_070-Variables8.csv',
   'EML1_018-Bias7.csv',
   'EML1_033-Validity1.csv',
   'EML1_069-Hypotheses5.csv',
   'EML1_025-CausalClaims6.csv',
   'EML1_125-CausalClaims6.csv',
   'EML1_020-Hypotheses1.csv',
   'EML1_111-CausalClaims6.csv',
   'EML1_048-Validity6.csv',
   'EML1_021-CausalClaims2.csv',
   'EML1_029-Variables5.csv',
   'EML1_083-Hypotheses8.csv',
   'EML1_015-Validity6.csv',
   'EML1_115-Validity6.csv',
   'EML1_136-Bias6.csv',
   'EML1_005-CausalClaims8.csv',
   'EML1_042-Validity6.csv',
   'EML1_103-CausalClaims8.csv',
   'EML1_106-Bias2.csv',
   'EML1_071-CausalClaims6.csv',
   'EML1_050-Variables8.csv',
   'EML1_072-Bias6.csv',
   'EML1_050-Bias2.csv',
   'EML1_012-Hypotheses8.csv',
   'EML1_030-Hypotheses8.csv',
   'EML1_036-Hypotheses4.csv',
   'EML1_036-CausalClaims6.csv',
   'EML1_014-CausalClaims3.csv',
   'EML1_036-Bias6.csv',
   'EML1_087-Hypo

In [22]:
load_file_folds("/Users/rickgentry/emotive_lab/eyemind/data/inference_train_val_rotex_folds.yml")

[(['EML1_087-Bias8.csv',
   'EML1_049-Validity8.csv',
   'EML1_069-Variables3.csv',
   'EML1_116-Hypotheses8.csv',
   'EML1_070-Variables8.csv',
   'EML1_018-Bias7.csv',
   'EML1_033-Validity1.csv',
   'EML1_069-Hypotheses5.csv',
   'EML1_025-CausalClaims6.csv',
   'EML1_125-CausalClaims6.csv',
   'EML1_020-Hypotheses1.csv',
   'EML1_111-CausalClaims6.csv',
   'EML1_048-Validity6.csv',
   'EML1_021-CausalClaims2.csv',
   'EML1_029-Variables5.csv',
   'EML1_083-Hypotheses8.csv',
   'EML1_015-Validity6.csv',
   'EML1_115-Validity6.csv',
   'EML1_136-Bias6.csv',
   'EML1_005-CausalClaims8.csv',
   'EML1_042-Validity6.csv',
   'EML1_103-CausalClaims8.csv',
   'EML1_106-Bias2.csv',
   'EML1_071-CausalClaims6.csv',
   'EML1_050-Variables8.csv',
   'EML1_072-Bias6.csv',
   'EML1_050-Bias2.csv',
   'EML1_012-Hypotheses8.csv',
   'EML1_030-Hypotheses8.csv',
   'EML1_036-Hypotheses4.csv',
   'EML1_036-CausalClaims6.csv',
   'EML1_014-CausalClaims3.csv',
   'EML1_036-Bias6.csv',
   'EML1_087-Hypo

In [23]:
# compare_splits("/Users/rickgentry/emotive_lab/eyemind/data/inference_train_val_rotex_folds.yml", file_folds, types="folds")
# compare_splits("/Users/rickgentry/emotive_lab/eyemind/data/inference_train_test_rotex_splits.yml", train_test_split)

### Loading Folds

In [24]:
configpath = "/Users/rickgentry/emotive_lab/eyemind/experiment_configs/fix_multiseq_config.yml"
with open(configpath, 'r') as f:
    config = yaml.safe_load(f)

In [25]:
name_to_cls = {"trainer": Trainer, "model": VariableSequenceLengthEncoderDecoderModel, "data": SequenceToSequenceDataModule}
trainer, model, data = instantiate_lightningmodules(config)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CNN Layers: [16, 32]
16 2
32 16


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [26]:
data.load_setup_path = "setup_ROTE_X.yml"
data.setup()

In [27]:
data.train_dataset

In [28]:
data.load_folds("folds_rote_x.yml")

In [31]:
num_folds = len(data.splits)

In [34]:
default_fit_loop = trainer.fit_loop
trainer.fit_loop = KFoldLoop(num_folds)
trainer.fit_loop.connect(default_fit_loop)

In [35]:
trainer

In [36]:
trainer.fit(model, datamodule=data)


  | Name            | Type             | Params
-----------------------------------------------------
0 | encoder         | Sequential       | 163 K 
1 | decoder         | RNNDecoder       | 198 K 
2 | criterion       | CrossEntropyLoss | 0     
3 | auroc_metric    | AUROC            | 0     
4 | accuracy_metric | Accuracy         | 0     
-----------------------------------------------------
362 K     Trainable params
0         Non-trainable params
362 K     Total params
1.448     Total estimated model params size (MB)


AttributeError: 'SequenceToSequenceDataModule' object has no attribute 'label_col'

### Test Metrics

In [ ]:
data.setup()

In [ ]:
res = trainer.test_loop.run()

In [ ]:
res

In [ ]:
label_df = pd.read_csv("/Users/rickgentry/emotive_lab/eyemind/data/EML1_pageLevel_with_filename.csv", index_col=0)

In [ ]:
from eyemind.dataloading.load_dataset import get_filenames_for_dataset, label_files
label_df = pd.read_csv("/Users/rickgentry/emotive_lab/eyemind/data/EML1_pageLevel_with_filename_seq.csv")
files1 = sorted(get_filenames_for_dataset("/Users/rickgentry/emotive_lab/eyemind/data/processed/output", label_df, "Rote_X"))
labels = label_files(label_df, "Rote_X", files1)
print(list(zip(files1, labels)))
label_df[["Rote_X","filename"]].loc[label_df["filename"].isin([file.split(".")[0] for file in files1])]


In [ ]:
new_df = add_sequence_col(label_df, "/Users/rickgentry/emotive_lab/eyemind/data/processed/output")

In [ ]:
new_df.to_csv("/Users/rickgentry/emotive_lab/eyemind/data/EML1_pageLevel_with_filename_seq.csv")

In [ ]:
new_df.loc[(~new_df["Rote_X"].isna()) & (new_df["sequence_length"] >500)]

In [ ]:
rote_x_label_df = new_df.loc[(~new_df["Rote_X"].isna()) & (new_df["sequence_length"] >500)]

In [ ]:
rote_x_label_df['Rote_X'].value_counts()

In [ ]:
312/894


In [ ]:
import torch


In [ ]:
a = torch.arange(40).reshape(4,5,2)


In [ ]:
a

In [ ]:
X = a.split(2, dim=1)[:-1]
print(X)

In [ ]:
torch.stack(X,1)[0].shape

In [ ]:
from eyemind.obf.model import creator
import torch

In [ ]:
# Have to add path to enable torch.load to work since they saved it weirdly
import sys
from pathlib import Path
sys.path.append(str(Path("../obf").resolve()))

In [ ]:
enc = creator.load_encoder("/Users/rickgentry/emotive_lab/eyemind/OBF/pre_weights/sample_weights/encoder_1633040995_gru.pt")

In [ ]:
enc

In [ ]:
torch.save(enc.state_dict(), "/Users/rickgentry/emotive_lab/eyemind/pretrained_models/obf_encoder_conv_gru.pt")

In [ ]:
from eyemind.models.classifier import create_encoder

encoder = create_encoder()

In [ ]:
encoder.load_state_dict(torch.load("/Users/rickgentry/emotive_lab/eyemind/pretrained_models/obf_encoder_conv_gru.pt"))

In [ ]:
encoder.parameters()

In [ ]:
for p_1, p_2 in zip(encoder.parameters(),enc.parameters()):
    assert torch.equal(p_1,p_2)